In [1]:

!pip install  numpy -q
!pip install  pandas -q
!pip install scikit_learn -q
# !pip install seaborn -q
!pip install torch -q
!pip install torchvision -q


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
#reading data
df = pd.read_csv("/content/data.csv")

In [4]:
df.head()

,year,customer_id,phone_no,gender,age,no_of_days_subscribed,multi_screen,mail_subscribed,weekly_mins_watched,minimum_daily_mins,maximum_daily_mins,weekly_max_night_mins,videos_watched,maximum_days_inactive,customer_support_calls,churn
0,2015,100198,409-8743,Female,36,62,no,no,148.35,12.2,16.81,82,1,4.0,1,0.0
1,2015,100643,340-5930,Female,39,149,no,no,294.45,7.7,33.37,87,3,3.0,2,0.0
2,2015,100756,372-3750,Female,65,126,no,no,87.30,11.9,9.89,91,1,4.0,5,1.0
3,2015,101595,331-4902,Female,24,131,no,yes,321.30,9.5,36.41,102,4,3.0,3,0.0
4,2015,101653,351-8398,Female,40,191,no,no,243.00,10.9,27.54,83,7,3.0,1,0.0


In [5]:
df = df.drop(["year", "customer_id", "phone_no"], axis=1)

In [6]:
print(df.shape)
print(df.columns)
df.dtypes

(2000, 13)
Index(['gender', 'age', 'no_of_days_subscribed', 'multi_screen',
       'mail_subscribed', 'weekly_mins_watched', 'minimum_daily_mins',
       'maximum_daily_mins', 'weekly_max_night_mins', 'videos_watched',
       'maximum_days_inactive', 'customer_support_calls', 'churn'],
      dtype='object')


gender                     object
age                         int64
no_of_days_subscribed       int64
multi_screen               object
mail_subscribed            object
weekly_mins_watched       float64
minimum_daily_mins        float64
maximum_daily_mins        float64
weekly_max_night_mins       int64
videos_watched              int64
maximum_days_inactive     float64
customer_support_calls      int64
churn                     float64
dtype: object

In [7]:
final_data = df.dropna()
final_data.shape

(1918, 13)

In [8]:
final_data.isnull().sum()

gender                    0
age                       0
no_of_days_subscribed     0
multi_screen              0
mail_subscribed           0
weekly_mins_watched       0
minimum_daily_mins        0
maximum_daily_mins        0
weekly_max_night_mins     0
videos_watched            0
maximum_days_inactive     0
customer_support_calls    0
churn                     0
dtype: int64

In [9]:
final_data["churn"].value_counts()

0.0    1665
1.0     253
Name: churn, dtype: int64

In [10]:
df_majority = final_data[final_data['churn']==0] #class 0
df_minority = final_data[final_data['churn']==1] #class 1

In [11]:
df_minority_upsampled = resample(df_minority, replace=True, n_samples=900, random_state=123) #upsampling minority class
df_majority_downsampled = resample(df_majority, replace=False, n_samples=900, random_state=123) #downsampling majority class

In [12]:
df_minority_upsampled.shape, df_majority_downsampled.shape

((900, 13), (900, 13))

In [13]:
#concanating both upsampled and downsampled class
df2 = pd.concat([df_majority_downsampled, df_minority_upsampled])

In [14]:
df2["churn"].value_counts()

0.0    900
1.0    900
Name: churn, dtype: int64

In [15]:
df2[['gender', 'multi_screen', 'mail_subscribed']].sample(2)

,gender,multi_screen,mail_subscribed
827,Male,no,no
1701,Male,no,no


In [16]:
#label encoding categorical variables
label_encoder = preprocessing.LabelEncoder()
df2['gender']= label_encoder.fit_transform(df2['gender'])
df2['multi_screen']= label_encoder.fit_transform(df2['multi_screen'])
df2['mail_subscribed']= label_encoder.fit_transform(df2['mail_subscribed'])

In [17]:
df2.head()

,gender,age,no_of_days_subscribed,multi_screen,mail_subscribed,weekly_mins_watched,minimum_daily_mins,maximum_daily_mins,weekly_max_night_mins,videos_watched,maximum_days_inactive,customer_support_calls,churn
1813,1,35,148,1,0,222.30,8.6,25.19,62,3,3.0,2,0.0
1362,1,42,98,0,0,241.50,12.1,27.37,113,4,4.0,4,0.0
389,0,39,56,0,0,379.80,4.4,43.04,133,4,2.0,1,0.0
1203,1,42,37,0,0,350.55,12.0,39.73,101,2,4.0,2,0.0
1710,1,44,63,0,0,246.75,11.2,27.97,97,2,4.0,0,0.0


In [18]:
#indenpendent variable
X = df2.iloc[:,:-1]

#output column
Y = df2["churn"]

In [19]:
#scaling the data
# Bringing the mean to 0 and variance to 1, so as to have a non-noisy optimization


sc = StandardScaler()
X = sc.fit_transform(X)
X = sc.transform(X)

In [20]:
n_samples, n_features = X.shape

In [21]:
# import sklearn
from sklearn.model_selection import train_test_split

In [27]:
#splitting data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42, stratify = Y)

In [28]:
print((y_train == 1).sum())
print((y_train == 0).sum())

630
630


In [29]:
print(type(X_train))
print(type(X_test))
print(type(y_train.values))
print(type(y_test.values))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [30]:
# convert np array to the tensor as PyTorch works on Tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))

y_train = torch.from_numpy(y_train.values.astype(np.float32))
y_test = torch.from_numpy(y_test.values.astype(np.float32))

In [31]:
# vector Y as a column vector for matrix multiplications

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [32]:
#logistic regression class
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    #sugmoid transformation of the input
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [33]:
lr = LogisticRegression(n_features)

## SGD optimizer

In [35]:
num_epochs = 500
# Traning the model for large number of epochs to see better results
learning_rate = 0.01
# We are working on lgistic regression so using Binary Cross Entropy
criterion = nn.BCELoss()

# Using SGD optimizer to find local minima
optimizer = torch.optim.SGD(lr.parameters(), lr=learning_rate)


In [36]:
for epoch in range(num_epochs):
    y_pred = lr(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1) % 20 == 0:
        # printing loss values on every 10 epochs to keep track
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 20, loss = 0.7310
epoch: 40, loss = 0.6962
epoch: 60, loss = 0.6760
epoch: 80, loss = 0.6637
epoch: 100, loss = 0.6556
epoch: 120, loss = 0.6498
epoch: 140, loss = 0.6454
epoch: 160, loss = 0.6419
epoch: 180, loss = 0.6389
epoch: 200, loss = 0.6363
epoch: 220, loss = 0.6339
epoch: 240, loss = 0.6317
epoch: 260, loss = 0.6296
epoch: 280, loss = 0.6277
epoch: 300, loss = 0.6259
epoch: 320, loss = 0.6241
epoch: 340, loss = 0.6224
epoch: 360, loss = 0.6208
epoch: 380, loss = 0.6193
epoch: 400, loss = 0.6178
epoch: 420, loss = 0.6163
epoch: 440, loss = 0.6149
epoch: 460, loss = 0.6136
epoch: 480, loss = 0.6123
epoch: 500, loss = 0.6111


In [37]:
with torch.no_grad():
    y_predicted = lr(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.6519


## ADAM optimizer

In [38]:
# Using ADAM optimizer to find local minima

num_epochs = 500
# Traning the model for large number of epochs to see better results
learning_rate = 0.01

# We are working on lgistic regression so using Binary Cross Entropy
criterion = nn.BCELoss()

optimizer = torch.optim.Adam(lr.parameters(), lr=learning_rate)



for epoch in range(num_epochs):
    y_pred = lr(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1) % 20 == 0:
        # printing loss values on every 10 epochs to keep track
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 20, loss = 0.5972
epoch: 40, loss = 0.5868
epoch: 60, loss = 0.5799
epoch: 80, loss = 0.5754
epoch: 100, loss = 0.5724
epoch: 120, loss = 0.5703
epoch: 140, loss = 0.5688
epoch: 160, loss = 0.5674
epoch: 180, loss = 0.5662
epoch: 200, loss = 0.5651
epoch: 220, loss = 0.5639
epoch: 240, loss = 0.5628
epoch: 260, loss = 0.5617
epoch: 280, loss = 0.5605
epoch: 300, loss = 0.5594
epoch: 320, loss = 0.5583
epoch: 340, loss = 0.5571
epoch: 360, loss = 0.5560
epoch: 380, loss = 0.5550
epoch: 400, loss = 0.5539
epoch: 420, loss = 0.5528
epoch: 440, loss = 0.5518
epoch: 460, loss = 0.5508
epoch: 480, loss = 0.5498
epoch: 500, loss = 0.5489


In [39]:
with torch.no_grad():
    y_predicted = lr(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.6852


In [40]:
# Looking other Metrics: We can also see the precision, recall, and F1-score using classification report

#classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted_cls))

              precision    recall  f1-score   support

         0.0       0.66      0.77      0.71       270
         1.0       0.72      0.60      0.66       270

    accuracy                           0.69       540
   macro avg       0.69      0.69      0.68       540
weighted avg       0.69      0.69      0.68       540



In [41]:
#confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_predicted_cls)
print(confusion_matrix)

[[207  63]
 [107 163]]


https://towardsdatascience.com/logistic-regression-with-pytorch-3c8bbea594be

In [ ]:
# logistic regression model
import torch
class LogisticRegression(torch.nn.Module):
     def __init__(self, input_dim, output_dim):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(input_dim, output_dim)
     def forward(self, x):
         outputs = torch.sigmoid(self.linear(x))
         return outputs

